## Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

In [31]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [32]:
#!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup
import requests
page = requests.get(url)

In [33]:
#page.text
soup = BeautifulSoup(page.text,'html.parser')
print(soup.title)
print(soup.title.string)

<title>List of postal codes of Canada: M - Wikipedia</title>
List of postal codes of Canada: M - Wikipedia


In [85]:
import pandas as pd
table = soup.find('table', {"class":"wikitable sortable"}).tbody
row = table.find_all('tr')
colnames = [v.text.replace('\n','') for v in row[0].find_all('th')]
tds = row.find_all('td')
#print(table)
print(len(row))

181


#### Web scraping table from wiki with bf4

In [106]:
#print(soup.a)
#print(soup.p)
#print(soup.p.text)
#print(soup.p.parent)
import pandas as pd
table = soup.find('table', {"class":"wikitable sortable"}).tbody
row = table.find_all('tr')
colnames = [v.text.replace('\n','') for v in row[0].find_all('th')]


df = pd.DataFrame(columns=colnames)
for i in range(1,len(row)):
    tds = row[i].find_all('td')
    
    if len(tds)==2:
        values = [tds[0].text.replace('\n',''),tds[1].text.replace('\n',''),''.replace('\n','')]
    else:
        values = [td.text.replace('\n','') for td in tds]
        
    df = df.append(pd.Series(values, index=colnames), ignore_index=True)
    df.to_csv('canadaneighbor.csv', index=False)

In [15]:
# try using Pandas
#!conda install -c conda-forge pandas.io.html --yes
#!conda install -c conda-forge lxml --yes
#!conda install -c conda-forge html5lib --yes
# import lxml
# from pandas.io.html import read_html
# wikitable = read_html(url, attrs={"class":"wikitable"})
# print('Extract {} table from wikipedia'.format(len(wikitable)))#

In [2]:
import pandas as pd
can = pd.read_csv('canadaneighbor.csv').dropna(how='any',axis=0).reset_index(drop=True)
can = pd.DataFrame(can) # convert target dataset into pandas data frame
can.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
#!conda install -c conda-forge geopy --yes
#from geopy import geocoder # import geocoder

from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("North York")
print(location.address)
print((location.latitude, location.longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


North York, Toronto, Golden Horseshoe, Ontario, Canada
(43.7543263, -79.44911696639593)


In [16]:
cor = pd.read_csv('GC.csv')
#print(cor.columns)
can_cordinated = pd.merge(can,cor[['Postal Code', 'Latitude', 'Longitude']],
                           on = 'Postal Code', how = 'left')

can_cordinated.to_csv('can_cordinated.csv', index=False)

In [17]:
can_cordinated = pd.read_csv('can_cordinated.csv').dropna(how='any',axis=0).reset_index(drop=True)
can_cordinated = pd.DataFrame(can_cordinated) # convert target dataset into pandas data frame
can_cordinated.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
